## Requirements
On importe toutes les librairies nécéssaires

In [1]:
import numpy as np
import re
import pandas as pd
#!pip install unidecode
from unidecode import unidecode
import torch

In [2]:
from PyPDF2 import PdfReader

In [3]:
import os
pd.set_option('mode.chained_assignment', None)
import s3fs

In [4]:
import matplotlib.pyplot as plt

## Fonctions

In [5]:
def normalize_text(s):
    """removing caractères, /, accents, m3, /n"""
    import string, re
    
    def no_unit(text):
        regex="m3"
        return re.sub(regex, " ", text)
    def no_exp(text):
        regex="m³"
        return re.sub(regex, " ", text)
    def no_punct(text):
        regex="\\.(?!\d)"
        return re.sub(regex," ",text)
   
    def white_space_fix(text):
        return " ".join(text.split())
    def no_date(text):
        date_extract_pattern = "[0-9]{1,2}\\/[0-9]{1,2}\\/[0-9]{4}"
        return re.sub(date_extract_pattern,"",text)
    def special_char(text):
        regex=regex = "[^\w\s,.]"
        return re.sub(regex, " ", text)
   #on retire tout les caractère des réponses 
    def no_char(text):
        regex=r"([a-zA-Z])"
        return re.sub(regex, " ", text)
    def comma(text):
        regex=",(?!\d)"
        return re.sub(regex," ",text)
    def point(text):
        return re.sub("\,",".",text)
    def indic(text):
        regex="(P\d\d\d\\.\d\s*)|(D\d\d\d\\.\d\s*)|(VP\\.\d\d\d)"
        return re.sub(regex," ", text)

    return white_space_fix(point(no_punct(no_char(unidecode(comma(special_char(indic(no_exp(no_unit(no_date(s)))))))))))

In [6]:
def normalize(s):
    """removing caractères, /, accents, m3, /n"""
    import string, re
    def no_punct(text):
        regex="\\.(?!\d)"
        return re.sub(regex," ",text)
    def white_space_fix(text):
        return " ".join(text.split())
    def special_char(text):
        regex=regex = "[^\w\s,.]"
        return re.sub(regex, " ", text)
    def comma(text):
        regex=",(?!\d)"
        return re.sub(regex," ",text)
    def lower(text):
        return text.lower()

    return white_space_fix(no_punct(unidecode(lower(special_char(s)))))

In [7]:
def white_space_fix(text):
        return " ".join(text.split())

In [8]:
def lower(text):
        return text.lower()

In [9]:
def saut(text):
        regex="\n"
        return re.sub(regex,"",text)

In [10]:
def compute_f1(prediction, truth):
    pred_tokens =normalize(prediction).split()
    truth_tokens = normalize(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    #print(len(common_tokens))
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

## Charger les modèles
On charge le modèle de question réponse d'etalab pour appliquer les pré-filtres, puis le modèle T5 pour récupérer les valeurs d'indicateurs.

In [11]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
from transformers import pipeline
nlp = pipeline('question-answering', model='etalab-ia/camembert-base-squadFR-fquad-piaf', tokenizer='etalab-ia/camembert-base-squadFR-fquad-piaf')

## Charger les données de SISPEA
On charge le documents contenant toutes les questions: associées à un mot clé et à un indicateur

In [595]:
FILE_KEY_Q = "camemBERT/data_2021_EP.xlsx" #Nom du fichier chargé depuis sispea

In [596]:
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "/mberthe/narval"
FILE_PATH_Q = BUCKET + "/" + FILE_KEY_Q

with fs.open(FILE_PATH_Q,  mode="rb") as file_in:
       df = pd.read_excel(file_in)


In [597]:
for i in df.index:
    df["Nom collectivité"][i]=lower(df["Nom collectivité"][i])

In [598]:
df_publi = df

### Charger les questions

In [63]:
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "/mberthe/narval"
FILE_KEY_Q = "camemBERT/question_all.csv"
FILE_PATH_Q = BUCKET + "/" + FILE_KEY_Q

with fs.open(FILE_PATH_Q,  mode="r") as file_in:
    question=pd.read_csv(file_in)

In [64]:
question

,question,mot,indic
0,Quel est le prix de service au m3 pour l'eau p...,prix,D102.0
1,Quel est le prix du service au m3 pour l'assin...,prix,D204.0
2,Quelle est la connaissance du réseau en eau po...,connaissance,P103.2B
3,Quel est le taux de connaissance du réseau en ...,connaissance,P202.2B
4,Quel est le taux de renouvellement des réseaux...,renouvellement,P107.2
...,...,...,...
97,Quelle est la valeur de l'endettement du servi...,dette,P256.2
98,Quelle est la valeur de l'endettement du servi...,endettement,P256.2
99,Quelle est la valeur de l'endettement du servi...,dette,P153.2
100,Quel est le total de points de connaissance d...,connaissance,P202.2B


## Pré-filtres
On récupère les premieres pages du rapport, desquelles on extrait l'année du rapport aisni que le type de service proposé 

In [686]:
nom_fichier='PDF/estal_EP_2021.pdf'

In [687]:
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "mberthe/narval"
fs.ls(BUCKET)
BUCKET = "mberthe/narval"
FILE_KEY_S3 = nom_fichier #entrer le nom du PDF
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    reader = PdfReader(file_in)
    page=reader.pages
    text=[]
    for i in range(0,5):
        page = reader.pages[i]
        output = page.extract_text()
        text.append(output)
    

In [688]:
text1=''
for x in text:
    text1 +=' '+ x

In [689]:
#text1

#### Le type de service

In [690]:
reau= "eau potable"
rass="assainissement"

d=nlp({'question': "Quels types de services ? ",
         'context': text1})
print(d)
prediction=d["answer"]
prediction=normalize(prediction)
if ((re.findall(reau,prediction)==[])&(re.findall(rass,prediction)==[])):
    d=nlp({'question': "eau potable ou assainissement collectif ? ",
         'context': text1})
    print(d)
    prediction=d["answer"]
    prediction=normalize(prediction)
    if ((re.findall(reau,prediction)==[])&(re.findall(rass,prediction)==[])):
        d=nlp({'question': "Quelle est l'entité de gestion ? ",
         'context': text1})
        print(d)
        prediction=d["answer"]
        prediction=normalize(prediction)
    if ((re.findall(reau,prediction)==[])&(re.findall(rass,prediction)==[])):
            print("type de service non retrouvé dans le texte")

{'score': 4.05071368732024e-05, 'start': 4, 'end': 26, 'answer': '\nEstal  \n \neau potable'}


Grâce à notre réponse sur le service, et la métrique f1, on choisit la liste d'indicateur appropriée

In [691]:
truth="""eau potable"""
prediction=d['answer']
print(prediction)
f1_ep=compute_f1(prediction,truth)
print('f1 eau potable:',f1_ep)

truth="""assainissement"""
prediction=normalize(d['answer'])
f1_ac=compute_f1(prediction,truth)
print('f1 assainissement:',f1_ac)

if ((re.findall(reau,prediction)!=[])&(re.findall(rass,prediction)!=[])):
    print(re.findall(reau,prediction),re.findall(rass,prediction))
    indic=["D102.0","D204.0","P103.2","P202.2","P107.2","P253.2","P109.0","P207.0","P153.2","P256.2","P154.0","P257.0","P155.1","P258.1","P101.1","P102.1","P104.3","P105.3","P106.3","P108.3","P151.1","D203.0","P201.1","P204.3","P205.3","P206.3","P251.1","P252.2","P254.3","P255.3","D302.0","P301.3"]
    service="AC+EP"
else:
    if (f1_ep > f1_ac ):
        indic=["D102.0","P103.2B","P107.2","P109.0","P153.2","P154.0","P155.1","P101.1","P102.1","P104.3","P105.3","P106.3","P108.3","P151.1"]
        service="EP"
    if (f1_ep < f1_ac):
        indic=["D204.0","P202.2B","P253.2","P207.0","P256.2","P257.0","P258.1","D203.0","P201.1","P204.3","P205.3","P206.3","P251.1","P252.2","P254.3","P255.3","D302.0","P301.3"]
        service="AC"
if(f1_ep==0)&(f1_ac==0):
         indic=["D102.0","D204.0","P103.2B","P202.2B","P107.2","P253.2","P109.0","P207.0","P153.2","P256.2","P154.0","P257.0","P155.1","P258.1","P101.1","P102.1","P104.3","P105.3","P106.3","P108.3","P151.1","D203.0","P201.1","P204.3","P205.3","P206.3","P251.1","P252.2","P254.3","P255.3","D302.0","P301.3"]
         service="Not defined"
print(indic)
print(service)


Estal  
 
eau potable
f1 eau potable: 0.8
f1 assainissement: 0
['D102.0', 'P103.2B', 'P107.2', 'P109.0', 'P153.2', 'P154.0', 'P155.1', 'P101.1', 'P102.1', 'P104.3', 'P105.3', 'P106.3', 'P108.3', 'P151.1']
EP


#### L'année du rapport

In [692]:
année=nlp({'question': "Quelle est l'année d'exercice du rapport ?",
         'context': text1})
print(année)

{'score': 0.8883946537971497, 'start': 127, 'end': 132, 'answer': ' 2021'}


In [693]:
an=normalize_text(année['answer']) # on normalise la réponse afin de n'avoir que l'année 

#### La ville
Pour comparer avec les résultats prédits

In [694]:
d_ville=nlp({'question': "Quel est le nom de la ville?",
         'context': text1})
ville1=d_ville['answer']
print(ville1)

 Estal


In [695]:
d_ville=nlp({'question': "Quel est le nom de la collectivité?",
         'context': text1})
ville2=d_ville['answer']
print(ville2)

 Estal


In [696]:
cols=df.columns

In [697]:
df_ville=pd.DataFrame(columns=[cols])
f1=0
row_list=[]
ind=''
for i in df_publi.index:
    if normalize(df["Nom collectivité"][i])==normalize(ville2):
        print('ville trouvée',i,df["Nom collectivité"][i],ville1)
        row_list.append(df_publi.loc[i])
df_ville=pd.DataFrame(row_list)
ville=normalize(ville1)
print(ville)
if df_ville.empty:
    ('empty2')
    for i in df_publi.index:
            #print(type(f1))
            if compute_f1(df["Nom collectivité"][i],ville) > f1:
                print(i)
                ind=i 
                f1=compute_f1(df["Nom collectivité"][i],ville)
    if not ind:
        print('ville not found')
    else:
        row_list.append(df_publi.loc[ind])
        df_ville=pd.DataFrame(row_list)
        print(df_publi["Nom collectivité"][ind])
        print(ind)
            
        

ville trouvée 4299 estal  Estal
estal


In [698]:
df_ville

,DPT du siège de la coll.,Id SISPEA de la collectivité,Nom collectivité,Courriels des utilisateurs collectivité,Type collectivité,N° SIREN,N° INSEE si commune,Communes membres de la coll,Avec ou sans CCSPL,Production,...,P105.3,P106.3,P107.2,P108.3,P109.0,P151.1,P152.1,P153.2,P154.0,P155.1
4299,046,169529,estal,estal.mairie@wanadoo.fr,Commune,214600975.0,46097,1,Non,Non,...,0.1,NaN,0.0,100.0,0.0,0.0,100.0,NaN,0.0,0.0


In [699]:
df_ville.empty

False

## On charge tout le texte du PDF
On va maintenant extraire les valeurs des RPQS

In [401]:
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "mberthe/narval"
fs.ls(BUCKET)
BUCKET = "mberthe/narval"
FILE_KEY_S3 = nom_fichier # entrer le nom du PDF
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    reader = PdfReader(file_in)
    page=reader.pages
    count = len(reader.pages)
    print(count)
    text=[]
    for i in range(count):
        page = reader.pages[i]
        output = page.extract_text()
        text.append(output)

20


In [402]:
pages =[]
for i in question.index:
    my_list=[]
    if question["indic"][i] in indic:
        for j in text:
            #print(re.findall(f1["mot"][i],j))
                if (re.findall(question["mot"][i],j)!=[]):
                    my_list.append(j)
                    print("question:",i,"page",text.index(j))
            #print(my_list)
    pages.append(my_list)
#len(pages[2])  

question: 1 page 0
question: 1 page 11
question: 3 page 1
question: 3 page 12
question: 3 page 13
question: 3 page 19
question: 5 page 1
question: 5 page 13
question: 9 page 2
question: 9 page 14
question: 9 page 15
question: 9 page 17
question: 11 page 1
question: 11 page 10
question: 11 page 19
question: 13 page 12
question: 13 page 13
question: 13 page 19
question: 15 page 1
question: 21 page 1
question: 21 page 10
question: 21 page 19
question: 23 page 12
question: 23 page 13
question: 23 page 19
question: 25 page 1
question: 36 page 17
question: 38 page 0
question: 38 page 11
question: 40 page 1
question: 40 page 12
question: 40 page 13
question: 40 page 19
question: 42 page 1
question: 42 page 13
question: 49 page 1
question: 49 page 18
question: 49 page 19
question: 53 page 1
question: 57 page 1
question: 59 page 1
question: 61 page 1
question: 63 page 1
question: 65 page 1
question: 72 page 1
question: 72 page 2
question: 72 page 8
question: 72 page 11
question: 72 page 15
ques

## Prédictions:

On pose maintenant toutes les questions

In [386]:
pred=[]
for i in question.index:
    print("question :",i)
    list_pred=[]
    for j in pages[i]:
        print("page:",text.index(j))
        input_text = 'contexte: '+ j + 'question: ' + question['question'][i] + "en "+ an + " ?"
        input_ids = tokenizer(input_text, max_length=300,
            truncation="only_second",
            padding="max_length",return_tensors="pt").input_ids.to('cuda')
        outputs = model.generate(input_ids)
        list_pred.append(tokenizer.decode(outputs[0]))
        #print(tokenizer.decode(outputs[0]))
    pred.append(list_pred)

question : 0
question : 1
page: 0


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
pred1=question
pred1["pred"]=pred

In [ ]:
for i in pred1.index :
     pred1["pred"][i]=list(map(normalize_text, pred1["pred"][i]))
pred1.head(20)

In [ ]:
for i in pred1.index :
     pred1["pred"][i]=list(map(normalize_text, pred1["pred"][i]))
#pred1.head(20)

In [ ]:
#on affiche que les réponses prédites et pas les vides
for i in pred1.index:
    if not (pred1["pred"][i]):
        pred1=pred1.drop([i])

In [ ]:
#pred1.head(15)

In [ ]:
#indic

In [123]:
table=pd.DataFrame()
for i in indic:
    my_list=[]
    for j in pred1.index:
        if (pred1["indic"][j]==i):
            my_list=my_list + pred1["pred"][j]
    #print(i,my_list)
    d={"indic":i,"valeur":my_list}
    #print(d)
    df_dictionary = pd.DataFrame([d])
    table = pd.concat([table, df_dictionary], ignore_index=True)
print(table.head())
table.head()    

NameError: name 'pred1' is not defined

In [124]:
for l in table.index:
    for i in table['valeur'][l]:
        if (re.findall('\d+',i)==[]):
            table['valeur'][l].remove(i)
for l in table.index:
    for i in table['valeur'][l]:        
        if (i==''):
            table['valeur'][l].remove(i)

In [191]:
for l in table.index:
    for i in table["valeur"][l]:
        i = [item.split(' ') for item in i]
#pred

In [192]:
BUCKET_OUT = "mberthe/narval"
FILE_KEY_OUT_S3 = "camemBERT/pre_" + nom_fichier + "_T5_xl.csv" #modif non du csv
FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
    table.to_csv(file_out,index=False)

# Reprendre

In [700]:
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "mberthe/narval"
FILE_KEY_S3 ="camemBERT/pre_" + nom_fichier + "_T5_xl.csv" #modif nom du csv 
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb",index= False) as file_in:
    pred = pd.read_csv(file_in)

In [701]:
import ast

In [702]:
pred

,indic,valeur
0,D102.0,"['0', '2.31', '______', '2.31', '0']"
1,P103.2B,"['1.6.1', '80 80', '0', '0', '80 80', '0', '0'..."
2,P107.2,"['0.1', '0', '0 0', '2021', '0', '0 0', '0', '..."
3,P109.0,"['5.1', '0', '0 0', '0', '0 0']"
4,P153.2,['0']
5,P154.0,"['3.8', '3.8', '3.8']"
6,P155.1,['3.9']
7,P101.1,"['100', '100 100', '0', '100 100', '0', '100 1..."
8,P102.1,"['85.7', '85.7 100', '2021', '0', '85.7', '0',..."
9,P104.3,"['92.5 92.4', '0', '92.5 92.4', '92.5 92.4', '..."


In [703]:
type(pred["valeur"][0])

str

In [704]:
#Nous avons un string donc, on le met sous forme de liste
for i in pred.index:
   pred['valeur'][i] =res = ast.literal_eval(pred['valeur'][i])

In [705]:
for i in pred.index:
    pred['valeur'][i]= [item.split(' ') for item in pred['valeur'][i]]
#pred

In [706]:
for l in pred.index:
    temp=[]
    for i in pred['valeur'][l]:
        for j in i:
            print(j)
            temp.append(j)
    pred['valeur'][l]=temp
#pred

0
2.31
______
2.31
0
1.6.1
80
80
0
0
80
80
0
0
80
80
0
80
3.2

0
0
0.1
0
0
0
2021
0
0
0
0
0
0
3.4
0.1
0
0
0
5.1
0
0
0
0
0
0
0
3.8
3.8
3.8
3.9
100
100
100
0
100
100
0
100
100
100
100
100
100
85.7
85.7
100
2021
0
85.7
0
85.7
85.7
100
85.7
100
92.5
92.4
0
92.5
92.4
92.5
92.4
92.5
92.4
92.5
92.4
92.5
92.4
9
____
0
____
____
9
92.5
92.4
0.1
0
0.1
0.1
0.1
0
0
0
____
0
____
____
0
92.5
0.1
100
0.1
442
0
92.5
92.4
0.1
0
0.1
0.1
0.1
0
0
92.5
0.1
0.1
0.1
3.4
0
100
100
3.4
0
100


In [707]:
import re
for l in pred.index:
    my_list=[]
    for i in pred['valeur'][l]:
       if (re.findall('\d+',i)==[]):
            my_list.append(i)
    for i in my_list:
        pred['valeur'][l].remove(i)

In [708]:
reg=r"\d\.(?!0)"

for i in pred.index:
    if pred["indic"][i] in ["P103.2","P202.2","P254.3","P255.3"]:
        print(pred['valeur'][i])
        temp=[] 
        for x in pred['valeur'][i]:
            print(x)
            if (re.findall(reg,x)!=[]):
                temp.append(x)
                print('ajoute x à temp:', x)
        for x in temp:
            pred['valeur'][i].remove(x)
            print('retire :',x)
        print(pred['valeur'][i])

In [709]:
pred

,indic,valeur
0,D102.0,"[0, 2.31, 2.31, 0]"
1,P103.2B,"[1.6.1, 80, 80, 0, 0, 80, 80, 0, 0, 80, 80, 0,..."
2,P107.2,"[0.1, 0, 0, 0, 2021, 0, 0, 0, 0, 0, 0, 3.4, 0...."
3,P109.0,"[5.1, 0, 0, 0, 0, 0, 0]"
4,P153.2,[0]
5,P154.0,"[3.8, 3.8, 3.8]"
6,P155.1,[3.9]
7,P101.1,"[100, 100, 100, 0, 100, 100, 0, 100, 100, 100,..."
8,P102.1,"[85.7, 85.7, 100, 2021, 0, 85.7, 0, 85.7, 85.7..."
9,P104.3,"[92.5, 92.4, 0, 92.5, 92.4, 92.5, 92.4, 92.5, ..."


In [710]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False


In [711]:
for i in pred.index:
    pred['valeur'][i]=[float(x) for x in pred['valeur'][i] if isfloat(x)]


In [712]:
pred

,indic,valeur
0,D102.0,"[0.0, 2.31, 2.31, 0.0]"
1,P103.2B,"[80.0, 80.0, 0.0, 0.0, 80.0, 80.0, 0.0, 0.0, 8..."
2,P107.2,"[0.1, 0.0, 0.0, 0.0, 2021.0, 0.0, 0.0, 0.0, 0...."
3,P109.0,"[5.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,P153.2,[0.0]
5,P154.0,"[3.8, 3.8, 3.8]"
6,P155.1,[3.9]
7,P101.1,"[100.0, 100.0, 100.0, 0.0, 100.0, 100.0, 0.0, ..."
8,P102.1,"[85.7, 85.7, 100.0, 2021.0, 0.0, 85.7, 0.0, 85..."
9,P104.3,"[92.5, 92.4, 0.0, 92.5, 92.4, 92.5, 92.4, 92.5..."


In [713]:
for i in pred.index:
    pred["valeur"][i]=set(pred["valeur"][i])

## Ajout des bornes

In [714]:
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "mberthe/narval"
FILE_KEY_S3 = "camemBERT/bornes.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb",index=False) as file_in:
    bornes = pd.read_csv(file_in)

In [715]:
#bornes

In [716]:
pred['min']=""
pred['max']=""
for i in bornes.index:
    for j in pred.index:
        if bornes['indic'][i]==pred['indic'][j]:
            pred["min"][j]=float(bornes['min'][i])
            pred['max'][j]=float(bornes['max'][i])

In [717]:
for i in pred.index:
    temp=[]
    for x in pred['valeur'][i]:
        print(pred['indic'][i],x,type(x),pred['min'][i],type(pred['min'][i]))
        if (float(x)<pred['min'][i]) or (float(x)>pred['max'][i]):
            temp.append(x)
    for x in temp:
        pred['valeur'][i].remove(x)

D102.0 0.0 <class 'float'> 1.25 <class 'float'>
D102.0 2.31 <class 'float'> 1.25 <class 'float'>
P103.2B 80.0 <class 'float'> 39.0 <class 'float'>
P103.2B 0.0 <class 'float'> 39.0 <class 'float'>
P103.2B 3.2 <class 'float'> 39.0 <class 'float'>
P107.2 0.1 <class 'float'> 0.0 <class 'float'>
P107.2 0.0 <class 'float'> 0.0 <class 'float'>
P107.2 3.4 <class 'float'> 0.0 <class 'float'>
P107.2 2021.0 <class 'float'> 0.0 <class 'float'>
P109.0 0.0 <class 'float'> 0.0 <class 'float'>
P109.0 5.1 <class 'float'> 0.0 <class 'float'>
P153.2 0.0 <class 'float'> 0.0 <class 'float'>
P154.0 3.8 <class 'float'> 0.0 <class 'float'>
P155.1 3.9 <class 'float'> 0.0 <class 'float'>
P101.1 0.0 <class 'float'> 83.3 <class 'float'>
P101.1 100.0 <class 'float'> 83.3 <class 'float'>
P102.1 0.0 <class 'float'> 78.9 <class 'float'>
P102.1 2021.0 <class 'float'> 78.9 <class 'float'>
P102.1 100.0 <class 'float'> 78.9 <class 'float'>
P102.1 85.7 <class 'float'> 78.9 <class 'float'>
P104.3 0.0 <class 'float'> 51.3 <

In [718]:
### €pred['indic'][1]="P103.2B"
pred

,indic,valeur,min,max
0,D102.0,{2.31},1.25,3.26
1,P103.2B,{80.0},39.0,120.0
2,P107.2,"{0.1, 0.0}",0.0,2.98
3,P109.0,{0.0},0.0,0.04
4,P153.2,{0.0},0.0,19.0
5,P154.0,{3.8},0.0,8.5
6,P155.1,{},0.0,2.01
7,P101.1,{100.0},83.3,100.0
8,P102.1,"{100.0, 85.7}",78.9,100.0
9,P104.3,"{92.5, 92.4}",51.3,97.3


In [719]:
print(ville)

estal


In [720]:
BUCKET_OUT = "mberthe/narval"
FILE_KEY_OUT_S3 = "camemBERT/rf_" + nom_fichier + "_T5_xl.csv"   #modif nom csv
FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
    pred.to_csv(file_out,index=False)

## Table des resultats finaux

In [721]:
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "mberthe/narval"
FILE_KEY_S3 = "camemBERT/indic_metric.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb",index=False) as file_in:
    indic_metric = pd.read_csv(file_in, sep= ';')

In [722]:
df_ville

,DPT du siège de la coll.,Id SISPEA de la collectivité,Nom collectivité,Courriels des utilisateurs collectivité,Type collectivité,N° SIREN,N° INSEE si commune,Communes membres de la coll,Avec ou sans CCSPL,Production,...,P105.3,P106.3,P107.2,P108.3,P109.0,P151.1,P152.1,P153.2,P154.0,P155.1
4299,046,169529,estal,estal.mairie@wanadoo.fr,Commune,214600975.0,46097,1,Non,Non,...,0.1,NaN,0.0,100.0,0.0,0.0,100.0,NaN,0.0,0.0


In [723]:
in_list=0
exact=0
for k in df_ville.index:
    print(k)
    in_list_temp=0
    exact_temp=0
    true_n=0
    false_p=0
    false_n=0
    for i in pred.index:
        for col in df_ville.columns:
            #for j in df_ville.index:
                if pred["indic"][i]==col:
                    df_ville[col][k]=round(df_ville[col][k],2)
                    print(col,'pred:',pred['valeur'][i],'true:',df_ville[col][k])
                    #Le nombre de bonnes réponses dans la liste des réponses prédites 
                    if df_ville[col][k] in pred["valeur"][i]:
                        in_list_temp=in_list_temp+1
                        print("in_list +1")
                        for indic in indic_metric.index:
                            if indic_metric["indic"][indic]==col:
                                indic_metric["in_list"][indic]=indic_metric["in_list"][indic] +1
                    #Le nombre de réponses exactes (true positive)    
                    if {df_ville[col][k]}== pred["valeur"][i]:
                        exact_temp=exact_temp+1
                        print("exact +1")
                        for indic in indic_metric.index:
                            if indic_metric["indic"][indic]==col:
                                indic_metric["exact"][indic]=indic_metric["exact"][indic] +1
                    #Le nombre de true negative (= pas de réponses retournée car pas de réponses dans le PDF)    
                    if np.isnan(df_ville[col][k]) & (pred['valeur'][i]== set()):
                        true_n=true_n+1
                        print('true_n +1')
                    #Le nombre de false positive(=Il y a une préiction alors qu'il n'y a pas de réponse dans le PDF)
                    if np.isnan(df_ville[col][k]) & (pred['valeur'][i]!= set()):
                        false_p= false_p+1
                        print('false_p +1')    
                    #Le nombre de false negative(=Il n'y a pas de predition alors qu'il y a une réponse dans le PDF)
                    if (not (np.isnan(df_ville[col][k]))) & (pred['valeur'][i]== set()):
                        false_n= false_n+1
                        print('false_n +1')    
                    
    print('code postal:',k,in_list_temp,exact_temp)
    if in_list_temp>in_list:
        in_list=in_list_temp
        exact=exact_temp
print('in_list:',in_list)
print('exact(=true_pos):',exact)
print('true_neg: ',true_n)
print('false_neg:',false_n)
print('false_pos:',false_p)

4299
D102.0 pred: {2.31} true: 2.31
in_list +1
exact +1
P103.2B pred: {80.0} true: 80.0
in_list +1
exact +1
P107.2 pred: {0.1, 0.0} true: 0.0
in_list +1
P109.0 pred: {0.0} true: 0.0
in_list +1
exact +1
P153.2 pred: {0.0} true: nan
false_p +1
P154.0 pred: {3.8} true: 0.0
P155.1 pred: set() true: 0.0
false_n +1
P101.1 pred: {100.0} true: 100.0
in_list +1
exact +1
P102.1 pred: {100.0, 85.7} true: 100.0
in_list +1
P104.3 pred: {92.5, 92.4} true: 92.4
in_list +1
P105.3 pred: {9.0} true: 0.1
P106.3 pred: set() true: nan
true_n +1
P108.3 pred: {100.0} true: 100.0
in_list +1
exact +1
P151.1 pred: set() true: 0.0
false_n +1
code postal: 4299 8 5
in_list: 8
exact(=true_pos): 5
true_neg:  1
false_neg: 2
false_pos: 1


In [724]:
indic_metric

,indic,in_list,exact
0,D102.0,14.0,7
1,P103.2,0.0,0
2,P107.2,19.0,0
3,P109.0,20.0,11
4,P153.2,0.0,0
5,P154.0,1.0,0
6,P155.1,0.0,0
7,P101.1,18.0,16
8,P102.1,20.0,14
9,P104.3,13.0,2


In [725]:
BUCKET_OUT = "mberthe/narval"
FILE_KEY_OUT_S3 = "camemBERT/indic_metric.csv"#modif nom csv
FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
    indic_metric.to_csv(file_out,index=False,sep=';')